In [1]:
import wmi
import gc
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    pid = process.ole_object.ProcessId
    del process
    del wmic
    gc.collect()

In [2]:
from pyMeow import open_process, get_module, r_float64, close_process, get_module, r_bytes, r_uint
import psutil

process = open_process(pid)
ps_util_process = psutil.Process(pid)
base_address = get_module(process, 'cloudmusic.dll')['base']

In [3]:
for offset in range(0xA70000, 0xC20000): # Change to estimated memory range
    found_songarray = False
    found_length = False
    found_current = False
    try:
        songid_array = r_uint(process, base_address + offset)
        song_id = r_bytes(process, songid_array, 0x14).decode('utf-16')
        if song_id == '1458767558':  # Change to current song ID
            print(f'found song array offset at {hex(offset)}: {song_id}')
            found_songarray = True
    except Exception as e:
        pass

    try:
        length = r_float64(process, base_address + offset)
        if length > 255 and length < 256:  # Change to song length, in seconds
            print(f'found length at {hex(offset)}: {length}')
            found_length = True
    except Exception as e:
        pass

    try:
        current = r_float64(process, base_address + offset)
        if current > 55 and current < 56:  # Change to current progress, in seconds
            print(f'found current at {hex(offset)}: {current}')
            found_current = True
    except Exception as e:
        pass
    if found_songarray and found_length and found_current:
        break

found current at 0xa79580: 55.561
found song array offset at 0xb2ad10: 1458767558
found length at 0xb2baf8: 255.24
found length at 0xbdf8b8: 255.27533486943685
